# 第四章：反思与博客写作

在这节课中，我们将了解智能体的反思框架，并利用它的力量创建高质量的博客文章，您将学习如何使用嵌套聊天对话模式去实现复杂的反思过程

我们的任务是去完成一个博客写作任务，具体来说，我们要创建一个系统，包含了一个评论智能体和一个写作智能体，评论智能体通过一些内嵌的审稿智能体来反思由写作代理撰写的博客文章。

# 一、环境配置

`autogen` 库我们装好了，启动环境后，我们需要定义 OpenAI 的 API Key 和模型配置信息：
  
（根据测试，`gpt-4o` 模型的效果会更好，且gpt-3.5-turbo的生成结果有一定随机性，在中英文环境下都有概率出错）

In [71]:
from utils import get_openai_api_key, get_openai_base_url
OPENAI_API_KEY = get_openai_api_key()
OPENAI_BASE_URL = get_openai_base_url()
llm_config = {"model": "gpt-3.5-turbo"}
# llm_config = {"model": "gpt-4o"} # 可以取消注释使用gpt-4o模型

# 二、任务定义

首先来定义我们的任务

In [41]:
task_zh = '''
        写一篇关于DeepLearning.AI的简洁但引人入胜的博客文章。
        确保博客文章是
        100词以内。
        '''

# 三、写作智能体创建

要完成这个任务，我们会自然想到需要建立一个写作智能体

In [42]:
import autogen

writer_zh = autogen.AssistantAgent(
    name="作家",
    system_message="你是一位作家. 你会关于给定的话题，写出简洁并且引人入胜的 " 
        "博客文章。 你必须根据你获得反馈润色你写出的文字，然后给出一个修改后的 "
        "版本。只返回你的最终作品，不需要任何额外的文字。",
    llm_config=llm_config,
)

[autogen.oai.client: 06-15 13:54:40] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


我们当然可以直接让写作智能体输出文章，但是这样可能效果不是最好的

In [43]:
reply_zh = writer_zh.generate_reply(messages=[{"content": task_zh, "role": "user"}])
print(reply_zh)

标题：探寻DeepLearning.AI的奥秘

在当今数码时代，人工智能是引领创新的主要驱动力之一。而DeepLearning.AI作为人工智能领域的重要一环，在深度学习技术中扮演着关键角色。从神经网络到自然语言处理，DeepLearning.AI让机器具备了人类般的智慧，为各行各业带来了巨大变革。通过专业的在线课程和实践项目，DeepLearning.AI为学习者提供了接触和掌握人工智能技术的独特机会。探索DeepLearning.AI，探寻人工智能的奥秘，引领未来的科技发展。


其实效果还不错，但是我们可以进一步优化它，一个思路就是用反思技巧，反思技巧是著名且有效的智能设计范式之一。

# 四、增加反思部分

加下来创建一个评论智能体来反思写作智能体的作品，我们通过自定义 `system_message` 从而操纵评论智能体 `critic` 来审查写作智能体 `writer` 的输出，并提供反馈。
  
我们这里使用的是 `autogen` 里的 `AsistantAgent` 类，`AsistantAgent` 类是我们熟悉的 `ConversableAgent` 类的子类，区别是 `AsistantAgent` 类配置了默认的 `system message` 。

In [44]:
critic_zh = autogen.AssistantAgent(
    name="评论家",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="你是一个评论家。你接受作家的作品，并且"
                "提供建设性的反馈，从而帮助提升内容的质量。",
)

[autogen.oai.client: 06-15 13:54:41] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


现在我们有了两个智能体，我们让他们之间发起对话。

In [45]:
res_zh = critic_zh.initiate_chat(
    recipient=writer_zh,
    message=task_zh,
    max_turns=2,
    summary_method="last_msg"
)

评论家 (to 作家):


        写一篇关于DeepLearning.AI的简洁但引人入胜的博客文章。
        确保博客文章是
        100词以内。
        

--------------------------------------------------------------------------------
作家 (to 评论家):

标题：探寻DeepLearning.AI的奥秘

在当今数码时代，人工智能是引领创新的主要驱动力之一。而DeepLearning.AI作为人工智能领域的重要一环，在深度学习技术中扮演着关键角色。从神经网络到自然语言处理，DeepLearning.AI让机器具备了人类般的智慧，为各行各业带来了巨大变革。通过专业的在线课程和实践项目，DeepLearning.AI为学习者提供了接触和掌握人工智能技术的独特机会。探索DeepLearning.AI，探寻人工智能的奥秘，引领未来的科技发展。

--------------------------------------------------------------------------------
评论家 (to 作家):

这篇博客文章开篇引人入胜，很好地探讨了DeepLearning.AI在人工智能领域的重要性和作用。要使文章更加生动，建议您在后续内容中增加一些具体的示例或案例，展示DeepLearning.AI在实际应用中的影响和成就。另外，可以提及一些学习者通过DeepLearning.AI课程取得的成就或成功故事，以吸引更多读者深入了解和学习。最后，可以将文章结尾处留下引人思考的问题或展望，为读者提供更多思考和探索的空间。希望这些建议能对您的博客文章有所帮助，期待看到更多精彩的内容！

--------------------------------------------------------------------------------
作家 (to 评论家):

标题：探寻DeepLearning.AI的奥秘

在当今数码时代，人工智能是引领创新的主要驱动力之一。DeepLearning.AI作为人工智能领域的关键之一，在各行各业发挥着无可比拟的作用。例如，医疗影像识别、智能交通系统和智能语音助手等领域的应用都凭借着D

通过对话历史可以看出，作家先给出了第一版的博客文章，然后评论家给了些反馈，最后作家又给了一版博客文章。  
  
从中我们可以看出，智能体在反思工作并且给出建议，但这还不够好，目前智能体给出的建议有些宽泛。  
  
有些情况下，我们可能会希望实现更加复杂的反思工作流来作为评论智能体的内心独白。例如，内容是否会在搜索搜索引擎中排名较高，目标读者的引流效果好不好，是否有法律和道德缺陷等。

# 五、NestedChat（嵌套聊天）

`NestedChat` 作为一个chat，被用作智能体的内心独白 (下图左下角灰色框中整体作为一个 NestedChat )  
  
  
![fig-4-1](./images/fig-4-1.png)

接下来，我们来定义这个chat，在博客写作任务中，我们可能需要很多审稿智能体来仔细监察作品的个个方面，所以我们先来创建一个SEO（Search Engine Optimization）智能体，即搜索引擎优化智能体。

In [53]:
SEO_reviewer_zh = autogen.AssistantAgent(
    name="SEO审稿人",
    llm_config=llm_config,
    system_message="你是一名SEO审稿人，以优化内容以适应搜索引擎的能力而闻名，"
        "确保内容在搜索引擎中的排名良好，并吸引优质流量。"
        "确保你的建议简洁（不超过3个要点）、"
        "具体且直截了当。"
        "在审查开始时，先陈述你的角色职责。",
)

[autogen.oai.client: 06-15 13:56:18] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


第二个审稿人是法律审稿人，我们需要他确保内容合法。  
  
最后一个审稿人是道德审稿人，它确保内容道德高尚且没有任何道德风险。

In [54]:
legal_reviewer_zh = autogen.AssistantAgent(
    name="法律审稿人",
    llm_config=llm_config,
    system_message="你是一名法律审稿人，负责确保内容符合法律规定并且没有任何潜在法律问题。"
        "确保你的建议简洁（不超过3个要点），"
        "具体且直截了当。"
        "在审查开始时，先陈述你的角色职责。",
)

[autogen.oai.client: 06-15 13:56:19] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [55]:
ethics_reviewer_zh = autogen.AssistantAgent(
    name="道德审稿人",
    llm_config=llm_config,
    system_message="你是一名伦理审稿人，负责确保内容在伦理上没有问题并且没有任何潜在伦理问题"
        "确保你的建议简洁（不超过3个要点），"
        "具体且直截了当。"
        "在审查开始时，先陈述你的角色职责。",
)

[autogen.oai.client: 06-15 13:56:19] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


我们还需要一名元审稿人（meta reviewer）来整理所有的评论并且给出最终建议。

In [56]:
meta_reviewer_zh = autogen.AssistantAgent(
    name="元审稿人",
    llm_config=llm_config,
    system_message="你是一名元审稿人，你汇总并审查其他审稿人的工作，"
    "并对内容提供最终建议。",
)

[autogen.oai.client: 06-15 13:56:20] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


# 六、将嵌套对话（Nested Chats）都编排起来解决问题

接下来我们来定义对话 ，我们使用上节课学的顺序聊天对话模式（Sequential Chat Conversation Pattern）来建立一系列评论家和审稿人之间的对话。这里的对话顺序是一个评论家分别与内部嵌套的审稿人对话的顺序结构，并非直觉上评论家发送一次消息后，所有审稿人同时作出相应的顺序。  
  
![fig-4-2](./images/fig-4-2.png)

如下所示，我们有一个包含四个对话的列表，每一个都包含一个特定的审稿人作为接收方，之后我们会把这个对话列表注册到评论家智能体中，由它作为默认的对话发起方，所以在对话列表中不需要再额外指定对话发起者 sender 。

下面我们具体看对话列表中对话的定义。  
  
在前三个对话中的每次对话中，接收方需要收到合适的初始消息（我们需要将评论家接收到的作家第一版本的作品作为上图中的 `message` 传给各个审稿人），一个常见做法是从外部对话中获取总结的消息，`reflection_message` 中的 `chat_messages_for_summary` 就是这个目的，函数内部的 `sender` 变量就对应评论家。  
  
同时，顺序对话范式还会通过“结转”机制（`Carryover`）将上一次聊天的摘要带入下一次聊天的上下文中，即之后对话中 `"Context:"` 后跟着的内容，且结转的内容会不断积累。
  
每个对话的输出部分我们希望用大模型来做格式化的总结（即以Json格式输出，包括一个审稿人字段和一个评论字段），所以我们设置 `"reflection_with_llm"` 与 `"summary_prompt"` 来格式化对话输出。

In [67]:
def reflection_message(recipient, messages, sender, config):
    return f'''审查以下内容。 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats_zh = [
    {
     "recipient": SEO_reviewer_zh, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "仅以JSON对象的形式返回审查结果："
        "{'审稿人': '', '评论': ''}。这里审稿人应该是你的角色",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer_zh, 
    "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "仅以JSON对象的形式返回审查结果："
        "{'审稿人': '', '评论': ''}。这里审稿人应该是你的角色",},
     "max_turns": 1},
    {"recipient": ethics_reviewer_zh, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "仅以JSON对象的形式返回审查结果："
        "{'审稿人': '', '评论': ''}。这里审稿人应该是你的角色"},
     "max_turns": 1},
     {"recipient": meta_reviewer_zh, 
      "message": "汇总所有审稿人的反馈，直接输出给写作的最终建议。", 
     "max_turns": 1},
]

终于，我们可以把上面定义的审稿对话定义为评论家的嵌套对话了，注意，我们将触发器 `trigger` 设置为作家，这意味着评论家每次接收到作家的消息，都会自动将消息路由到嵌套对话中去进行仔细的反思。

In [68]:
critic_zh.register_nested_chats(
    review_chats_zh,
    trigger=writer_zh,
)

配置到后，我们现在可以发起评论家和作家之间的对话了，和最开始的流程类似，不同之处就是我们在评论家中定义了嵌套聊天来做仔细的反思。

In [69]:
res = critic_zh.initiate_chat(
    recipient=writer_zh,
    message=task_zh,
    max_turns=2,
    summary_method="last_msg"
)

评论家 (to 作家):


        写一篇关于DeepLearning.AI的简洁但引人入胜的博客文章。
        确保博客文章是
        100词以内。
        

--------------------------------------------------------------------------------
作家 (to 评论家):

标题：探寻DeepLearning.AI的奥秘

在当今数码时代，人工智能是引领创新的主要驱动力之一。而DeepLearning.AI作为人工智能领域的重要一环，在深度学习技术中扮演着关键角色。从神经网络到自然语言处理，DeepLearning.AI让机器具备了人类般的智慧，为各行各业带来了巨大变革。通过专业的在线课程和实践项目，DeepLearning.AI为学习者提供了接触和掌握人工智能技术的独特机会。探索DeepLearning.AI，探寻人工智能的奥秘，引领未来的科技发展。

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
评论家 (to SEO审稿人):

审查以下内容。 
            

 标题：探寻DeepLearning.AI的奥秘

在当今数码时代，人工智能是引领创新的主要驱动力之一。而DeepLearning.AI作为人工智能领域的重要一环，在深度学习技术中扮演着关键角色。从神经网络到自然语言处理，DeepLearning.AI让机器具备了人类般的智慧，为各行各业带来了巨大变革。通过专业的在线课程和实践项目，DeepLearning.AI为学习者提供了接触和掌握人工智能技术的独特机会。探索DeepLearning.AI，探寻人工智能的奥秘，引领未来的科技发展。

----

通过上方的具体聊天记录可见：  
  
- 第一步是作家生成文章，然后评论家没有直接给出反馈，而是将消息路由到了嵌套对话中。  
  
- 然后是评论家和SEO审稿人的对话，SEO检查内容后，给出关于优化搜索引擎排名相关的建议，比如优化标题；

- 接下来是法律审稿人，检查法律风险并给出建议，比如不侵犯知识产权；之后是道德审稿人，给出建议如确保内容准确不夸大。  
  
- 最后是元审稿人，负责总结所有建议，并且通过评论家给到作家。  
    
- 作家根据意见修改，得到新一版的作品。
  
我们发现在嵌套对话中，之前的聊天信息作为结转（`carryover`）会在 `Content` 中在每次对话中给到接收方。

# 七、获得概述

进一步，我们调用结果的摘要属性 `suammary` 来检查结果，可见，它返回了最终修改后的博客文章。

In [66]:
print(res.summary)

标题：探索DeepLearning.AI：引领人工智能的革新之路

在当今数码时代，DeepLearning.AI作为人工智能领域的重要组成部分正引领着技术革新的潮流。融合神经网络、自然语言处理等技术，深度学习赋予机器以智能，改变着各行各业的运作方式。通过专业的在线课程和实践项目，DeepLearning.AI为学习者展开了人工智能的奥秘之旅，培养着未来的科技领袖。探索DeepLearning.AI，了解技术背后的原理与应用，助力科技迈向更辉煌的未来。


我们可以继续尝试不同的任务，比如写你感兴趣主题的文章。  
  

  
总结： 这节课中我们学习了如何利用嵌套聊天实现反思设计范式，下节课中，我们将进一步学习如何用嵌套聊天来提升智能体设计范式。